In [2]:
import pandas as pd

In [14]:
query = "SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` WHERE date BETWEEN '20170101' AND '20170131'"

In [15]:
query

"SELECT * FROM `bigquery-public-data.google_analytics_sample.ga_sessions_*` WHERE date BETWEEN '20170101' AND '20170131'"

In [16]:
df = pd.read_gbq(query=query, project_id='urm-kpi-br', dialect='standard', dtype={'visitId':object})

Requesting query... ok.
Job ID: 238cbc5d-6938-491f-b2f7-2e94ce6cdc5a
Query running...
  Elapsed 7.11 s. Waiting...
  Elapsed 8.16 s. Waiting...
  Elapsed 9.18 s. Waiting...
  Elapsed 10.38 s. Waiting...
  Elapsed 11.79 s. Waiting...
  Elapsed 12.83 s. Waiting...
  Elapsed 13.86 s. Waiting...
  Elapsed 15.11 s. Waiting...
  Elapsed 16.11 s. Waiting...
  Elapsed 17.14 s. Waiting...
  Elapsed 18.15 s. Waiting...
  Elapsed 19.18 s. Waiting...
  Elapsed 20.19 s. Waiting...
Query done.
Processed: 5.4 GB Billed: 5.4 GB
Standard price: $0.03 USD

Retrieving results...
Got 64694 rows.

Total time taken 1137.09 s.
Finished at 2022-09-14 17:21:41.


In [17]:
df.head()

visitorId  visitNumber     visitId  visitStartTime      date  \
0      None            1  1483857382      1483857382  20170107   
1      None            1  1483781643      1483781643  20170107   
2      None            1  1483791860      1483791860  20170107   
3      None            1  1483831862      1483831862  20170107   
4      None            1  1483856596      1483856596  20170107   

                                              totals  \
0  {'screenviews': None, 'uniqueScreenviews': Non...   
1  {'screenviews': None, 'uniqueScreenviews': Non...   
2  {'screenviews': None, 'uniqueScreenviews': Non...   
3  {'screenviews': None, 'uniqueScreenviews': Non...   
4  {'screenviews': None, 'uniqueScreenviews': Non...   

                                       trafficSource  \
0  {'adContent': None, 'campaign': '(not set)', '...   
1  {'adContent': None, 'campaign': '(not set)', '...   
2  {'adContent': None, 'campaign': '(not set)', '...   
3  {'adContent': None, 'campaign': '(not set)', '...   
4  {'adContent': None, 'campaign': '(not set)', '...   

                                              device  \
0  {'screenResolution': 'not available in demo da...   
1  {'screenResolution': 'not available in demo da...   
2  {'screenResolution': 'not available in demo da...   
3  {'screenResolution': 'not available in demo da...   
4  {'screenResolution': 'not available in demo da...   

                                          geoNetwork  \
0  {'subContinent': 'Northern America', 'cityId':...   
1  {'subContinent': 'Southern Asia', 'cityId': 'n...   
2  {'subContinent': 'South America', 'cityId': 'n...   
3  {'subContinent': 'Western Europe', 'cityId': '...   
4  {'subContinent': 'Northern America', 'cityId':...   

                           customDimensions  \
0  [{'index': 4, 'value': 'North America'}]   
1           [{'index': 4, 'value': 'APAC'}]   
2  [{'index': 4, 'value': 'South America'}]   
3           [{'index': 4, 'value': 'EMEA'}]   
4  [{'index': 4, 'value': 'North America'}]   

                                                hits        fullVisitorId  \
0  [{'experiment': [], 'time': 0, 'promotionActio...  1387446501958018844   
1  [{'experiment': [], 'time': 0, 'promotionActio...  8993417660422454202   
2  [{'experiment': [], 'time': 0, 'promotionActio...  3896365782347437567   
3  [{'experiment': [], 'time': 0, 'promotionActio...  0454906255427936283   
4  [{'experiment': [], 'time': 0, 'promotionActio...  9435629583163991867   

  userId clientId channelGrouping  socialEngagementType  
0   None     None        Referral  Not Socially Engaged  
1   None     None          Direct  Not Socially Engaged  
2   None     None  Organic Search  Not Socially Engaged  
3   None     None          Direct  Not Socially Engaged  
4   None     None     Paid Search  Not Socially Engaged

In [18]:
df.shape

(64694, 16)

In [19]:
productos_sesion = []
precios_sesion = []

for linea in df.hits:
  productos_hit = []
  precios_hit = []
  for hit in linea:
    for producto in hit['product']:
      productos_hit.append(producto['productSKU'])
      precios_hit.append(producto['productPrice'])
  productos_sesion.append(productos_hit)
  precios_sesion.append(sum(precios_hit))

In [20]:
len(productos_sesion)

64694

In [21]:
productos_sesion[365]

['GGOEYHPA003510',
 'GGOEYDHJ056099',
 'GGOEYHPA003610',
 'GGOEYOLR018699',
 'GGOEYFKQ020699',
 'GGOEYHPB072210',
 'GGOEYDHJ019399',
 'GGOEYOLR080599',
 'GGOEYOBR078599',
 'GGOEYOCR078099',
 'GGOEYOCR077799',
 'GGOEYOCR077399',
 'GGOEGAAX0318',
 'GGOEGAAX0325',
 'GGOEGAAX0317']

In [22]:
precios_sesion[365]

194850000

In [23]:
s = pd.Series(productos_sesion)

In [24]:
from sklearn.preprocessing import MultiLabelBinarizer

In [25]:
mlb = MultiLabelBinarizer()

In [26]:
df_productos = pd.DataFrame(mlb.fit_transform(s), columns=mlb.classes_, index=df.index)

In [27]:
df = df.join(df_productos)

In [28]:
df['precio'] = precios_sesion

In [29]:
df.head()

visitorId  visitNumber     visitId  visitStartTime      date  \
0      None            1  1483857382      1483857382  20170107   
1      None            1  1483781643      1483781643  20170107   
2      None            1  1483791860      1483791860  20170107   
3      None            1  1483831862      1483831862  20170107   
4      None            1  1483856596      1483856596  20170107   

                                              totals  \
0  {'screenviews': None, 'uniqueScreenviews': Non...   
1  {'screenviews': None, 'uniqueScreenviews': Non...   
2  {'screenviews': None, 'uniqueScreenviews': Non...   
3  {'screenviews': None, 'uniqueScreenviews': Non...   
4  {'screenviews': None, 'uniqueScreenviews': Non...   

                                       trafficSource  \
0  {'adContent': None, 'campaign': '(not set)', '...   
1  {'adContent': None, 'campaign': '(not set)', '...   
2  {'adContent': None, 'campaign': '(not set)', '...   
3  {'adContent': None, 'campaign': '(not set)', '...   
4  {'adContent': None, 'campaign': '(not set)', '...   

                                              device  \
0  {'screenResolution': 'not available in demo da...   
1  {'screenResolution': 'not available in demo da...   
2  {'screenResolution': 'not available in demo da...   
3  {'screenResolution': 'not available in demo da...   
4  {'screenResolution': 'not available in demo da...   

                                          geoNetwork  \
0  {'subContinent': 'Northern America', 'cityId':...   
1  {'subContinent': 'Southern Asia', 'cityId': 'n...   
2  {'subContinent': 'South America', 'cityId': 'n...   
3  {'subContinent': 'Western Europe', 'cityId': '...   
4  {'subContinent': 'Northern America', 'cityId':...   

                           customDimensions   ...   GGOEYHPA003510  \
0  [{'index': 4, 'value': 'North America'}]   ...                0   
1           [{'index': 4, 'value': 'APAC'}]   ...                0   
2  [{'index': 4, 'value': 'South America'}]   ...                0   
3           [{'index': 4, 'value': 'EMEA'}]   ...                0   
4  [{'index': 4, 'value': 'North America'}]   ...                0   

  GGOEYHPA003610 GGOEYHPB072210 GGOEYOBR078599 GGOEYOCR077399 GGOEYOCR077799  \
0              0              0              0              0              0   
1              0              0              0              0              0   
2              0              0              0              0              0   
3              0              0              0              0              0   
4              0              0              0              0              0   

   GGOEYOCR078099  GGOEYOLR018699  GGOEYOLR080599  precio  
0               0               0               0       0  
1               0               0               0       0  
2               0               0               0       0  
3               0               0               0       0  
4               0               0               0       0  

[5 rows x 1126 columns]

In [30]:
visitas = df.drop(['hits','customDimensions'], axis=1)

In [31]:
visitas.head()

visitorId  visitNumber     visitId  visitStartTime      date  \
0      None            1  1483857382      1483857382  20170107   
1      None            1  1483781643      1483781643  20170107   
2      None            1  1483791860      1483791860  20170107   
3      None            1  1483831862      1483831862  20170107   
4      None            1  1483856596      1483856596  20170107   

                                              totals  \
0  {'screenviews': None, 'uniqueScreenviews': Non...   
1  {'screenviews': None, 'uniqueScreenviews': Non...   
2  {'screenviews': None, 'uniqueScreenviews': Non...   
3  {'screenviews': None, 'uniqueScreenviews': Non...   
4  {'screenviews': None, 'uniqueScreenviews': Non...   

                                       trafficSource  \
0  {'adContent': None, 'campaign': '(not set)', '...   
1  {'adContent': None, 'campaign': '(not set)', '...   
2  {'adContent': None, 'campaign': '(not set)', '...   
3  {'adContent': None, 'campaign': '(not set)', '...   
4  {'adContent': None, 'campaign': '(not set)', '...   

                                              device  \
0  {'screenResolution': 'not available in demo da...   
1  {'screenResolution': 'not available in demo da...   
2  {'screenResolution': 'not available in demo da...   
3  {'screenResolution': 'not available in demo da...   
4  {'screenResolution': 'not available in demo da...   

                                          geoNetwork        fullVisitorId  \
0  {'subContinent': 'Northern America', 'cityId':...  1387446501958018844   
1  {'subContinent': 'Southern Asia', 'cityId': 'n...  8993417660422454202   
2  {'subContinent': 'South America', 'cityId': 'n...  3896365782347437567   
3  {'subContinent': 'Western Europe', 'cityId': '...  0454906255427936283   
4  {'subContinent': 'Northern America', 'cityId':...  9435629583163991867   

    ...   GGOEYHPA003510 GGOEYHPA003610 GGOEYHPB072210 GGOEYOBR078599  \
0   ...                0              0              0              0   
1   ...                0              0              0              0   
2   ...                0              0              0              0   
3   ...                0              0              0              0   
4   ...                0              0              0              0   

   GGOEYOCR077399  GGOEYOCR077799  GGOEYOCR078099  GGOEYOLR018699  \
0               0               0               0               0   
1               0               0               0               0   
2               0               0               0               0   
3               0               0               0               0   
4               0               0               0               0   

   GGOEYOLR080599  precio  
0               0       0  
1               0       0  
2               0       0  
3               0       0  
4               0       0  

[5 rows x 1124 columns]

In [32]:
diccionarios = ['device', 'trafficSource', 'geoNetwork', 'totals']

In [ ]:
import json

# Transformando las llaves de los diccionarios en nuevas columnas
for columna in diccionarios:
    visitas = visitas.join(pd.DataFrame([json.loads(json.dumps(linea)) for linea in visitas[columna]]),rsuffix=('_'+ columna))
visitas.drop(diccionarios, axis=1, inplace=True)


# Corrigiendo el formato de las columnas con valores cuantitativos
totals = df.totals[0].keys()
totals = list(totals)
for columna in totals:
    visitas[columna] = pd.to_numeric(visitas[columna])


# Limpiando los datos
visitas.drop('adwordsClickInfo',axis=1,inplace=True)
columna_na = []
for columna in visitas.columns:
    print(columna + ': ' + str(len(visitas[columna].unique())))
    if len(visitas[columna].unique()) == 1:
        columna_na.append(columna)
visitas.drop(columna_na,axis=1,inplace=True)

In [34]:
visitas.head()

visitNumber     visitId  visitStartTime      date        fullVisitorId  \
0            1  1483857382      1483857382  20170107  1387446501958018844   
1            1  1483781643      1483781643  20170107  8993417660422454202   
2            1  1483791860      1483791860  20170107  3896365782347437567   
3            1  1483831862      1483831862  20170107  0454906255427936283   
4            1  1483856596      1483856596  20170107  9435629583163991867   

  channelGrouping  10 52048  10 52147  10 52211  10 52213      ...       \
0        Referral         0         0         0         0      ...        
1          Direct         0         0         0         0      ...        
2  Organic Search         0         0         0         0      ...        
3          Direct         0         0         0         0      ...        
4     Paid Search         0         0         0         0      ...        

                          region      subContinent  bounces  hits  newVisits  \
0  not available in demo dataset  Northern America      1.0     1        1.0   
1                          Delhi     Southern Asia      1.0     1        1.0   
2  not available in demo dataset     South America      1.0     1        1.0   
3  not available in demo dataset    Western Europe      1.0     1        1.0   
4                     California  Northern America      1.0     1        1.0   

   pageviews  timeOnSite  totalTransactionRevenue  transactionRevenue  \
0        1.0         NaN                      NaN                 NaN   
1        1.0         NaN                      NaN                 NaN   
2        1.0         NaN                      NaN                 NaN   
3        1.0         NaN                      NaN                 NaN   
4        1.0         NaN                      NaN                 NaN   

   transactions  
0           NaN  
1           NaN  
2           NaN  
3           NaN  
4           NaN  

[5 rows x 1142 columns]

In [35]:
visitas.drop('transactionRevenue',axis=1,inplace=True)

In [36]:
visitas.drop('transactions',axis=1,inplace=True)

In [37]:
visitas.totalTransactionRevenue = visitas.totalTransactionRevenue / 1000000

In [38]:
# Creando lista con variables cuantitativas
cuant = list(set(totals) - set(columna_na) - set(['transactionRevenue','transactions']))

#Creando dataframe con los resultados cuantitativos de las sesiones
visitas_totals = visitas.groupby('fullVisitorId',as_index=False)[cuant].sum()

# Última visita
visitas_ultima = visitas.groupby('fullVisitorId',as_index=False)
visitas_ultima = visitas_ultima['visitNumber'].max()

# Combinación entre visitantes y visitas únicas
usuarios_visitas_unicos = visitas.drop_duplicates(subset=['fullVisitorId','visitNumber'])

# Dataframe usuarios con todos los usuarios únicos y su última visita
usuarios = pd.merge(visitas_ultima,usuarios_visitas_unicos,left_on=['fullVisitorId','visitNumber'],
                    right_on=['fullVisitorId','visitNumber'],how='left')

# Primera visita
visitas_primera = visitas.groupby('fullVisitorId',as_index=False)['visitNumber'].min()
visitas_primera.set_index('fullVisitorId',inplace=True)

# Dataframe usuarios con todos los usuarios unicos + su última visita + primera visita
usuarios = usuarios.join(visitas_primera,how='left',on='fullVisitorId',rsuffix='_primera')
usuarios = pd.merge(usuarios,usuarios_visitas_unicos,left_on=['fullVisitorId','visitNumber_primera'],
                    right_on=['fullVisitorId','visitNumber'],how='left',suffixes=('_ultima','_primera'))

# Dataframe usuarios con todos los usuarios unicos + su última visita + primera visita + sumatoria de las columnas cuant
usuarios = pd.merge(usuarios,visitas_totals,left_on=['fullVisitorId'],
                    right_on=['fullVisitorId'],how='left')

# Eliminando totales
for i in cuant:
    usuarios.drop(i+'_primera',axis=1,inplace=True)
    usuarios.drop(i+'_ultima',axis=1,inplace=True)

# Calculando el tiempo entre primera y última visita
usuarios['tiempo_visitas'] = usuarios.visitStartTime_ultima - usuarios.visitStartTime_primera

In [39]:
#Eliminando las columnas Ids
ids = ['fullVisitorId', 'visitId_ultima','visitId_primera']
usuarios.drop(ids,axis=1,inplace=True)

#Creando una variable 'y' con la columna respuesta
y = usuarios.totalTransactionRevenue.copy()
y.fillna(0,inplace=True)
y[y<0] = 0

#Creando una variable X con todas las variables menos la respuesta
X = usuarios.drop('totalTransactionRevenue',axis=1)


#Transformando las variables cualitativas en numeros
from sklearn.preprocessing import LabelEncoder
cuali = usuarios.dtypes[usuarios.dtypes == 'object'].keys()
for col in cuali:
    lbl = LabelEncoder()
    lbl.fit(list(X[col].values.astype('str')))
    X[col] = lbl.transform(list(X[col].values.astype('str')))

/usr/local/envs/py3env/lib/python3.5/site-packages/ipykernel/__main__.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [40]:
X.shape

(53041, 2271)

In [41]:
X.head()

visitNumber_ultima  visitStartTime_ultima  date_ultima  \
0                   1             1484597865           15   
1                   1             1485675517           27   
2                   1             1483595731            3   
3                   1             1483629067            4   
4                   1             1484513468           14   

   channelGrouping_ultima  10 52048_ultima  10 52147_ultima  10 52211_ultima  \
0                       7                0                0                0   
1                       7                0                0                0   
2                       6                0                0                0   
3                       4                0                0                0   
4                       4                0                0                0   

   10 52213_ultima  10 52223_ultima  10 52225_ultima       ...        \
0                0                0                0       ...         
1                0                0                0       ...         
2                0                0                0       ...         
3                0                0                0       ...         
4                0                0                0       ...         

   metro_primera  networkDomain_primera  region_primera  subContinent_primera  \
0             40                   1075             135                    17   
1             40                   2545             135                     6   
2             36                    892             128                    12   
3              0                   4503              49                    21   
4             40                   4644             135                    13   

   bounces  timeOnSite  newVisits  hits  pageviews  tiempo_visitas  
0      1.0         0.0        1.0     1        1.0               0  
1      1.0         0.0        1.0     1        1.0               0  
2      0.0        78.0        1.0     5        5.0               0  
3      1.0         0.0        1.0     1        1.0               0  
4      1.0         0.0        1.0     1        1.0               0  

[5 rows x 2271 columns]